Ноутбук запускался в колабе

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla T4 GPUs


In [2]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 225kB 23.5MB/s 
     |████████████████████████████████| 2.3MB 39.8MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 245kB 26.5MB/s 
     |████████████████████████████████| 112kB 56.9MB/s 
     |████████████████████████████████| 901kB 29.7MB/s 
     |████████████████████████████████| 3.3MB 42.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=13166628f5f1c8167756089dbbe2216be7fc1f6dded875a28a5b246ae26aa941
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [3]:
task = "ner"
batch_size = 8

# Data - RFQ

In [4]:
!gdown --id 1L6dd0FnYqgn-eoQ-gFnBiNji7R1ul9n_

Downloading...
From: https://drive.google.com/uc?id=1L6dd0FnYqgn-eoQ-gFnBiNji7R1ul9n_
To: /content/NER_RFQ_agg.csv
4.39MB [00:00, 23.2MB/s]


In [5]:
!gdown --id 1-5mE9XjocmyCKGlkpW1YGuCnNsGwQioD

Downloading...
From: https://drive.google.com/uc?id=1-5mE9XjocmyCKGlkpW1YGuCnNsGwQioD
To: /content/NER_RFQ_agg_train.csv
3.48MB [00:00, 30.7MB/s]


In [6]:
!gdown --id 1d4er4I7x4VIwy7BsWFpsPuC2z6aZ3P6s

Downloading...
From: https://drive.google.com/uc?id=1d4er4I7x4VIwy7BsWFpsPuC2z6aZ3P6s
To: /content/NER_RFQ_agg_test.csv
100% 909k/909k [00:00<00:00, 14.1MB/s]


In [7]:
import pandas as pd
data = pd.read_csv('NER_RFQ_agg.csv', converters={'tokens': eval, 'ner_tags': eval})

In [8]:
all_tags = set()
for _, row in data.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))

In [9]:
tag_to_idx = {t: i for i, t in enumerate(all_tags)}

In [10]:
label_list = {i: t for i, t in enumerate(all_tags)}

In [11]:
train = pd.read_csv('NER_RFQ_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_RFQ_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

In [12]:
train.ner_tags = train.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])
test.ner_tags = test.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])

In [13]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [14]:
from datasets import load_dataset, load_metric

In [15]:
datasets = load_dataset('pandas', data_files={'train' : 'train.pkl', 'test' : 'test.pkl'})

Using custom data configuration default-7e20830aed851dbf


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-7e20830aed851dbf/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 953
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 239
    })
})

# Tokenizer

In [17]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [19]:
label_all_tokens = False

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


Warning : есть последовательности больше, чем максимально возможная в BERT - 512

In [22]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count     953.000000
mean      613.048269
std       598.540485
min       123.000000
25%       337.000000
50%       482.000000
75%       713.000000
max      6873.000000
dtype: float64

In [23]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count     239.000000
mean      630.610879
std       557.334506
min       190.000000
25%       342.500000
50%       483.000000
75%       759.000000
max      6800.000000
dtype: float64

In [24]:
train_tokenized = tokenized_datasets["train"].to_pandas()
test_tokenized = tokenized_datasets["test"].to_pandas()

In [25]:
train_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123783R.msg,"[101, 11324, 94757, 68073, 10129, 119, 92287, ...","[-100, 2, 2, -100, -100, -100, -100, -100, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[name, 123783r.msg, yuliya, a., kondratova, <y..."


In [26]:
test_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123670R.msg,"[101, 11324, 96903, 48205, 10129, 119, 92287, ...","[-100, 2, 2, -100, -100, -100, -100, -100, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[name, 123670r.msg, =?koi8-r?b?68ht2nhoz9fbiod..."


In [27]:
CHUNCK_SIZE = 512

def lists_to_chuncks(row):
    n = CHUNCK_SIZE - 2
    new_row = row.copy()
    for key in ['attention_mask', 'input_ids', 'labels', 'token_type_ids']:
        FIRST = new_row[key][0]
        LAST = new_row[key][-1]
        new_row[key] = new_row[key][1:-1]
        new_row[key] = [[FIRST] + list(new_row[key][i:i + n]) + [LAST] for i in range(0, len(new_row[key]), n)]
    return new_row

In [28]:
train_tokenized = train_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
train_tokenized = train_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [29]:
test_tokenized = test_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
test_tokenized = test_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [30]:
train_tokenized.to_pickle('train_tokenized.pkl')
test_tokenized.to_pickle('test_tokenized.pkl')

In [31]:
tokenized_datasets = load_dataset('pandas', data_files={'train' : 'train_tokenized.pkl', 'test' : 'test_tokenized.pkl'})

Using custom data configuration default-ef84e9731321aac5


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-ef84e9731321aac5/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [32]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    1553.00000
mean      376.97038
std       152.54258
min         3.00000
25%       262.00000
50%       431.00000
75%       512.00000
max       512.00000
dtype: float64

In [33]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count    399.000000
mean     378.536341
std      146.408350
min       15.000000
25%      266.500000
50%      413.000000
75%      512.000000
max      512.000000
dtype: float64

Теперь все последовательности нужной длины

# Model

In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Warning : модель надо обучить на наших данных

In [35]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [37]:
metric = load_metric("seqeval")

In [38]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.065683,0.574534,0.663082,0.615641,0.979764
2,No log,0.053492,0.733607,0.801971,0.766267,0.986775
3,0.053900,0.049634,0.765523,0.839606,0.800855,0.988373


TrainOutput(global_step=585, training_loss=0.048743364138480946, metrics={'train_runtime': 569.3029, 'train_samples_per_second': 1.028, 'total_flos': 364871250082080.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2069282816, 'init_mem_gpu_alloc_delta': 709342720, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 145952768, 'train_mem_gpu_alloc_delta': 2196961280, 'train_mem_cpu_peaked_delta': 262381568, 'train_mem_gpu_peaked_delta': 6504170496})

In [41]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9883725459644749,
 'eval_f1': 0.8008547008547008,
 'eval_loss': 0.04963363707065582,
 'eval_mem_cpu_alloc_delta': 139264,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 394249728,
 'eval_precision': 0.7655228758169934,
 'eval_recall': 0.839605734767025,
 'eval_runtime': 14.9751,
 'eval_samples_per_second': 26.644}

In [42]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'GoodsString': {'f1': 0.8008547008547008,
  'number': 1116,
  'precision': 0.7655228758169934,
  'recall': 0.839605734767025},
 'overall_accuracy': 0.9883725459644749,
 'overall_f1': 0.8008547008547008,
 'overall_precision': 0.7655228758169934,
 'overall_recall': 0.839605734767025}

In [43]:
pd.DataFrame.from_dict(results).transpose().iloc[:-4, :-1]

,f1,number,precision
GoodsString,0.800855,1116.0,0.765523


In [44]:
pd.DataFrame.from_dict(results).transpose().iloc[-4:, 0].rename('result')

overall_precision    0.765523
overall_recall       0.839606
overall_f1           0.800855
overall_accuracy     0.988373
Name: result, dtype: float64

In [45]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 30.6MB/s 


In [46]:
from sklearn_crfsuite import metrics
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags[:-1], digits=3))

               precision    recall  f1-score   support

B-GoodsString      0.857     0.901     0.878      1109
I-GoodsString      0.888     0.875     0.882      1718

    micro avg      0.875     0.885     0.880      2827
    macro avg      0.873     0.888     0.880      2827
 weighted avg      0.876     0.885     0.880      2827

